In [76]:
!pip install polars beautifulsoup4 splinter selenium 

In [77]:
import polars as pl
from polars.exceptions import InvalidOperationError
import numpy as np
import pathlib
import re
from bs4 import BeautifulSoup
# Will use firefox browser
from splinter import Browser
import time

# Loading Data

In [78]:
cwd = pathlib.Path.cwd()

if cwd.name == 'Mild-Steel-Tempering':
    print("Path is project root")
else:
    print("Please correct current working directory to the project root")


Path is project root


In [79]:
resources_path = pathlib.PurePath(pathlib.PurePath(cwd), 'resources')
resources_path

PurePosixPath('/home/mox/Documents/coding_projects/bootcamp_local/Homeworks/Mild-Steel-Tempering/resources')

### Many alloy composition columns were parsed incorrectly and failing to load

Several weight percent columns were parsed as int automatically due to having "0" for many initial rows. 

All weight percent columns should be parsed as float. 

In [80]:
data_path = f"{resources_path}/Raiipa-tempering-data.csv"
schema_overrides = {"C (%wt)" : pl.Float64,
"Mn (%wt)" : pl.Float64,
"P (%wt)" : pl.Float64,
"S (%wt)" : pl.Float64,
"Si (%wt)" : pl.Float64,
"Ni (%wt)" : pl.Float64,
"Cr (%wt)" : pl.Float64,
"Mo (%wt)" : pl.Float64,
"V (%wt)" : pl.Float64,
"Al (%wt)" : pl.Float64,
"Cu (%wt)" : pl.Float64}

df_data = pl.read_csv(data_path, schema_overrides=schema_overrides)
df_data.glimpse()

Rows: 1466
Columns: 17
$ Source                                  <str> 'Grange and Baughman, 1956', 'Grange and Baughman, 1956', 'Grange and Baughman, 1956', 'Grange and Baughman, 1956', 'Grange and Baughman, 1956', 'Grange and Baughman, 1956', 'Grange and Baughman, 1956', 'Grange and Baughman, 1956', 'Grange and Baughman, 1956', 'Grange and Baughman, 1956'
$ Steel type                              <str> 'AISI-SAE 1026', 'AISI-SAE 1026', 'AISI-SAE 1026', 'AISI-SAE 1026', 'AISI-SAE 1026', 'AISI-SAE 1026', 'AISI-SAE 1026', 'AISI-SAE 1026', 'AISI-SAE 1026', 'AISI-SAE 1026'
$ Initial hardness (HRC) - post quenching <str> '?', '?', '?', '?', '?', '?', '?', '?', '?', '?'
$ Tempering time (s)                      <i64> 600, 600, 600, 600, 600, 600, 600, 600, 600, 600
$ Tempering temperature (ºC)              <f64> 204.4, 260.0, 315.6, 371.1, 426.7, 482.2, 537.8, 593.3, 648.9, 704.4
$ C (%wt)                                 <f64> 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25
$ Mn 

In [81]:
# save initial columns names as they are very descriptive and may be useful later
initial_column_names = df_data.columns
initial_column_names

['Source',
 'Steel type',
 'Initial hardness (HRC) - post quenching',
 'Tempering time (s)',
 'Tempering temperature (ºC)',
 'C (%wt)',
 'Mn (%wt)',
 'P (%wt)',
 'S (%wt)',
 'Si (%wt)',
 'Ni (%wt)',
 'Cr (%wt)',
 'Mo (%wt)',
 'V (%wt)',
 'Al (%wt)',
 'Cu (%wt)',
 'Final hardness (HRC) - post tempering']

# Cleaning
Many columns need renaming for ease of manipulation

Columns need datatypes correction
 'Initial hardness (HRC) - post quenching' needs datatype correction. ? is NA value


In [82]:
dict_new_cnames = {}
for og_name in initial_column_names:
    # replace filler in hardness columns
    new_name = og_name.replace(' - ', '')
    
    # Handle units
    try:
        #if alloy weight percent remove units and return only elemental symbol
        if re.search(r"\(%wt\)", og_name):
            new_name = og_name.split(' ')[0]
            #skip the rest of the try block that will re_add the units to the end
            pass
        else: 
            #If not elemental composition, lowercase the string
            new_name = new_name.lower()
        # regex find the units inside the parenthesis, of the original name, not the lowercased new name
        # This lines breaks and goes to except if there is no units
        unit = re.search(r'\((\w+)\)', og_name).group(1)
        # replace the unit parenthesis string with parenthesis with an empty string
        new_name = re.sub(r"\(.+\)", "", new_name)
        # trim to whitespace end characters left by some unit removals
        new_name = new_name.rstrip()
        # append the unit string to the end of the processed name
        new_name = f"{new_name}_{unit}"
    except:
        # skip unit processing on names with no units denoted by parenthesis
        pass

    # Strip away special characters
    new_name = new_name.encode("ascii", errors="ignore").decode()
    #replace all whitespace with underscores
    new_name = new_name.replace(' ', '_')
    #add the name to the rename dict
    dict_new_cnames[og_name] = new_name
dict_new_cnames


{'Source': 'source',
 'Steel type': 'steel_type',
 'Initial hardness (HRC) - post quenching': 'initial_hardness_post_quenching_HRC',
 'Tempering time (s)': 'tempering_time_s',
 'Tempering temperature (ºC)': 'tempering_temperature_C',
 'C (%wt)': 'C',
 'Mn (%wt)': 'Mn',
 'P (%wt)': 'P',
 'S (%wt)': 'S',
 'Si (%wt)': 'Si',
 'Ni (%wt)': 'Ni',
 'Cr (%wt)': 'Cr',
 'Mo (%wt)': 'Mo',
 'V (%wt)': 'V',
 'Al (%wt)': 'Al',
 'Cu (%wt)': 'Cu',
 'Final hardness (HRC) - post tempering': 'final_hardness_post_tempering_HRC'}

In [83]:
df_clean_cnames = df_data.rename(dict_new_cnames)

## Clean data types and column values

In [84]:
count_of_qmark = df_clean_cnames['initial_hardness_post_quenching_HRC'].value_counts()\
    .filter(pl.col('initial_hardness_post_quenching_HRC') == "?")\
        .select("count").item()


In [85]:
percent_intial_hardness_unknown = count_of_qmark / int(df_clean_cnames['initial_hardness_post_quenching_HRC'].shape[0])
print(f"Unknown initial hardness: {round(percent_intial_hardness_unknown, 2) * 100:.0f}%")


Unknown initial hardness: 65%


In [86]:
df_clean_cnames.tail(3)


source,steel_type,initial_hardness_post_quenching_HRC,tempering_time_s,tempering_temperature_C,C,Mn,P,S,Si,Ni,Cr,Mo,V,Al,Cu,final_hardness_post_tempering_HRC
str,str,str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Hollomon and Jaffe, 1945""","""1,15%C - plain carbon steel""","""64.5""",86400,500.0,1.15,0.58,0.012,0.021,0.09,0.0,0.01,0.0,0.0,0.0,0.0,32.0
"""Hollomon and Jaffe, 1945""","""1,15%C - plain carbon steel""","""64.5""",86400,600.0,1.15,0.58,0.012,0.021,0.09,0.0,0.01,0.0,0.0,0.0,0.0,23.0
"""Hollomon and Jaffe, 1945""","""1,15%C - plain carbon steel""","""64.5""",86400,700.0,1.15,0.58,0.012,0.021,0.09,0.0,0.01,0.0,0.0,0.0,0.0,4.5


In [87]:
df_clean_cnames["initial_hardness_post_quenching_HRC"].value_counts().sort('count', descending=True).head(3)

initial_hardness_post_quenching_HRC,count
str,u32
"""?""",949
"""66.5""",90
"""55.8""",51


In [88]:
df_clean = df_clean_cnames.with_columns(pl.col('initial_hardness_post_quenching_HRC').cast(pl.Float64, strict=False))


In [89]:
df_clean["initial_hardness_post_quenching_HRC"].value_counts().sort('count', descending=True).head(3)

initial_hardness_post_quenching_HRC,count
f64,u32
null,949
66.5,90
61.6,51


In [90]:
df_clean.describe()

statistic,source,steel_type,initial_hardness_post_quenching_HRC,tempering_time_s,tempering_temperature_C,C,Mn,P,S,Si,Ni,Cr,Mo,V,Al,Cu,final_hardness_post_tempering_HRC
str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""","""1466""","""1466""",517.0,1466.0,1466.0,1466.0,1466.0,1466.0,1466.0,1466.0,1466.0,1466.0,1466.0,1466.0,1466.0,1466.0,1466.0
"""null_count""","""0""","""0""",949.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",null,null,61.493617,21969.754434,422.024147,0.511583,0.74073,0.017236,0.023802,0.239379,0.362838,0.389696,0.080232,0.005457,0.034379,0.005986,41.468008
"""std""",null,null,5.656383,34177.623863,176.088041,0.224354,0.252913,0.007966,0.007967,0.239193,0.810091,0.480721,0.121422,0.02905,0.20534,0.019085,14.079248
"""min""","""Grange and Baughman, 1956""","""0,31%C - plain carbon steel""",46.5,10.0,100.0,0.25,0.3,0.007,0.005,0.06,0.0,0.0,0.0,0.0,0.0,0.0,0.9
"""25%""",null,null,58.8,600.0,260.0,0.37,0.6,0.012,0.018,0.16,0.0,0.02,0.0,0.0,0.0,0.0,32.1
"""50%""",null,null,63.1,3600.0,426.7,0.42,0.74,0.017,0.024,0.21,0.01,0.06,0.0,0.0,0.0,0.0,43.1
"""75%""",null,null,66.5,14400.0,593.3,0.56,0.8,0.019,0.029,0.24,0.06,0.8,0.22,0.0,0.0,0.0,51.8
"""max""","""Penha, 2010""","""Nitriding Steel """,67.0,115200.0,704.4,1.15,1.85,0.054,0.055,1.62,3.41,1.57,0.36,0.16,1.26,0.08,68.5


# Making steel types values searchable on AZoM.com
Steel types that do not include AISI and a code are not searchable in steel databased and need manual renaming

Exmaple: "0,74%C - plain carbon steel" is a AISI 1074 steel. Confirmed via elemental composition on https://www.azom.com/article.aspx?ArticleID=6558


In [91]:
steel_identifiers = ["steel_type", "source"]
df_clean[steel_identifiers].unique().shape

(36, 2)

## Renaming all possible steels so they are searchable

1945 source

I can not access source papers. Making a best guess of steel grade from elemental composition

I assume some alloying elements are a bit strange due to wartime shortages and potential contamination between alloys due to the push to increasew prodiction.

### additional Justifications are inline

In [92]:
#1945 source...
# I can not access source for the 1945 paper on these
# I assume they alloying elements are a bit strange due to wartime shortages
# and potentially contamination between alloys due to the push to increasew prodiction
dict_rename = {'0,98%C - plain carbon steel': 'AISI 1095', # copper and Cr, probably intentional?
                   '1,15%C - plain carbon steel': 'AISI 1095', # small chromium impurity?
                   '0,74%C - plain carbon steel': 'AISI 1074 Carbon Steel', 
                   '0,56%C - plain carbon steel': 'AISI 1055', # small chromium impurity?
                   '0,89%C - plain carbon steel': 'AISI 1090', # Cu discounted as impurity
                   'Nitriding Steel ': 'Non-searchable', 
                   '0,31%C - plain carbon steel': 'AISI 1030', # AISI 1030, Cr impurity
                   "AISI-SAE 9264" : 'AISI-SAE 9254', # access limited, may be AISI-SAE 9264 , not in AZoM
                   "AISI-SAE 2340" : 'Non-searchable', # access limited, may be SAE J2340, not in AZoM
                   "AISI-SAE 3140" : 'Non-searchable', # Not in AZoM elemental match to SAE 3140 https://www.steel-grades.com/metals/18/5155/-SAE-3140.html
                   "AISI-SAE 4068" : 'Non-searchable', # access limited, may be SAE 4068, not in AZoM
                   "AISI-SAE 4640" : "AISI 4640",
                   "AISI-SAE 4047" : "AISI 4047",
                   "AISI-SAE 1049" : "AISI 1049",
                   "AISI-SAE 6145" : "AISI 6145",
                   "AISI-SAE E52100" : "AISI 52100"} # # Not in AZoM elemental match to  SAE 4068https://www.steel-grades.com/Steel-Grades/Carbon-Steel/SAE-4068-.html

In [93]:
series_searchable_steel = df_clean.clone()
renamed = series_searchable_steel['steel_type'].replace(dict_rename)
series_searchable_steel = series_searchable_steel.with_columns(searchable = renamed)

In [94]:
series_to_search = series_searchable_steel['searchable'].unique()
steels_to_search = series_to_search.filter(series_to_search.eq('Non-searchable').not_())
steels_to_search

searchable
str
"""AISI-SAE 4037"""
"""AISI-SAE 1040"""
"""AISI-SAE 4140"""
"""AISI-SAE 1030"""
"""AISI-SAE 5160"""
…
"""AISI-SAE 1045"""
"""AISI-SAE 9254"""
"""AISI-SAE 5140"""


## Save Searchable steel
Checkpoint so that the code to this point does not need to be run every time

In [95]:
path_save_searchable = cwd / 'resources' / 'searchable_steels.csv'
pl.DataFrame(steels_to_search).write_csv(path_save_searchable)

# Web Scraping

Two Stages are necessary

AZoM does not have data on all steels.

MakeItFrom.com also does not have data from all steels. 

TODO: together they have data from ?? steels

In [96]:
len([print(steel) for steel in steels_to_search])

AISI-SAE 4037
AISI-SAE 1040
AISI-SAE 4140
AISI-SAE 1030
AISI-SAE 5160
AISI 1074 Carbon Steel
AISI 1055
AISI 4047
AISI 1030
AISI-SAE 1065
AISI 6145
AISI 1095
AISI-SAE 6150
AISI-SAE 1335
AISI-SAE 1038
AISI-SAE 4340
AISI 1049
AISI 52100
AISI 1090
AISI-SAE 1026
AISI-SAE 4027
AISI-SAE 1050
AISI-SAE 1042
AISI-SAE 1035
AISI-SAE 1045
AISI-SAE 9254
AISI-SAE 5140
AISI-SAE 1080
AISI 4640


29

## Scraping AZoM.com

In [97]:
browser = Browser('firefox')
base_url = "https://www.azom.com"
search_path = "/search.aspx?q="

In [98]:
# Search the steel 
def get_soup(soup):
    search_result = soup.find('div', class_='resultsContainer')
    first_item = search_result.find('a')
    # https://pytutorial.com/get-element-href-beautifulsoup/
    steel_link = first_item.get('href')
    time.sleep(0.15)
    browser.visit(f"{base_url}{steel_link}")
    soup2 = BeautifulSoup(browser.html, 'html.parser')
    return soup2


In [99]:
# already have elemental composition
# property - metric - imperial
# table_elements = tables[0]
# table_phys_props = tables[1]
# table_mecha_props = tables[2]
# table_therm_props = tables[3]
# table_other_desigs = tables[4]
#parse tables
def get_tables(soup):
    tables = []
    # parse all tables in the
    for html_table in soup.find_all('table'):
        table = []
        # parse rows in the table
        for tr in html_table.find_all('tr'):
            row = []
            #parse data cells in the row
            for t in tr.find_all(['th', 'td']):
                text = t.get_text(strip=True)
                #add data to row
                row.append(text)
            #add row to table
            table.append(row)
    #returns a list of list of lists
        tables.append(table)
    return tables
    

In [100]:
def make_dfs(lolols):
    tables = []
    # convert the list of lists to a dataframe
    for lol in lolols:
        try:
            df = pl.DataFrame(lol, orient='row')
            # rename columns using first row
            # https://stackoverflow.com/questions/75187317/how-to-rename-column-names-with-first-row-in-polars
            # [1:] removes the first row that replaced the column names 
            df = df.rename(df.head(1).to_dicts().pop())[1:]
            # add the dataframe to the list of tables on for this steel
            tables.append(df)
        except:
            tables.append(lol)
    return tables

In [101]:
dict_steel_tables = {}
for steel in steels_to_search:
    try:
        print("Working on:", steel)
        #search AZoM for the steel
        browser.visit(f"{base_url}{search_path}{steel}")
        soup_search = BeautifulSoup(browser.html, 'html.parser')
    
        #takes soup html and returns list of lists
        soup = get_soup(soup_search)
        lolols = get_tables(soup)
        dict_steel_tables[steel] = make_dfs(lolols)
        # tables list of lists and returns a dictionary
        # robots.txt indicates Crawl-delay: 120
        time.sleep(0.15)
    except: 
        dict_steel_tables[steel] = []

Working on: AISI-SAE 4037
Working on: AISI-SAE 1040
Working on: AISI-SAE 4140
Working on: AISI-SAE 1030
Working on: AISI-SAE 5160
Working on: AISI 1074 Carbon Steel
Working on: AISI 1055
Working on: AISI 4047
Working on: AISI 1030
Working on: AISI-SAE 1065
Working on: AISI 6145
Working on: AISI 1095
Working on: AISI-SAE 6150
Working on: AISI-SAE 1335
Working on: AISI-SAE 1038
Working on: AISI-SAE 4340
Working on: AISI 1049
Working on: AISI 52100
Working on: AISI 1090
Working on: AISI-SAE 1026
Working on: AISI-SAE 4027
Working on: AISI-SAE 1050
Working on: AISI-SAE 1042
Working on: AISI-SAE 1035
Working on: AISI-SAE 1045
Working on: AISI-SAE 9254
Working on: AISI-SAE 5140
Working on: AISI-SAE 1080
Working on: AISI 4640


In [102]:
dict_steel_tables

{'AISI-SAE 4037': [shape: (7, 2)
  ┌────────────────┬─────────────┐
  │ Element        ┆ Content (%) │
  │ ---            ┆ ---         │
  │ str            ┆ str         │
  ╞════════════════╪═════════════╡
  │ Iron, Fe       ┆ 97.98-98.6  │
  │ Manganese, Mn  ┆ 0.70-0.90   │
  │ Carbon, C      ┆ 0.35-0.40   │
  │ Silicon, Si    ┆ 0.15-0.35   │
  │ Molybdenum, Mo ┆ 0.20-0.30   │
  │ Sulfur, S      ┆ 0.040       │
  │ Phosphorous, P ┆ 0.035       │
  └────────────────┴─────────────┘,
  shape: (1, 3)
  ┌────────────┬────────────┬──────────────┐
  │ Properties ┆ Metric     ┆ Imperial     │
  │ ---        ┆ ---        ┆ ---          │
  │ str        ┆ str        ┆ str          │
  ╞════════════╪════════════╪══════════════╡
  │ Density    ┆ 7.85 g/cm3 ┆ 0.284 lb/in³ │
  └────────────┴────────────┴──────────────┘,
  shape: (10, 3)
  ┌─────────────────────────────────┬─────────────┬─────────────────┐
  │ Properties                      ┆ Metric      ┆ Imperial        │
  │ ---               

# Converting AZoM Scraped Data to one dataframe

## Working with list of tables

Must confirma all tables were sucesffuly made into polars tables

Must reduce the numebr of tables to only those we are concerned with

* Remove "Other Designations" tables, including those that did not get sucessfully converted to a polars dataframe (table 4)

* Remove tables with properties that are not commonly present (thermal propterties table 3)

* Remove elemental compositions, already present in the data (table 0)

* Physical and mechanical properties are desired (tables 1,2)

In [103]:
dict_steel_tables_keep = {}
bool_check = True
polars_check = True 
for steel, tables in dict_steel_tables.items():
    all_polars = True
    count_polars = 0
    for table in tables:
        if isinstance(table, pl.DataFrame): 
            count_polars += 1 
    if count_polars != len(tables):
        all_polars = False
    if not (bool_check and all_polars):
        bool_check = False
    print("All polars?", all_polars, '|', len(tables), "Tables | Steel:", steel)
    dict_steel_tables_keep[steel] = tables[1:3]
print(polars_check)

All polars? True | 4 Tables | Steel: AISI-SAE 4037
All polars? True | 4 Tables | Steel: AISI-SAE 1040
All polars? True | 5 Tables | Steel: AISI-SAE 4140
All polars? True | 5 Tables | Steel: AISI-SAE 1030
All polars? True | 4 Tables | Steel: AISI-SAE 5160
All polars? True | 3 Tables | Steel: AISI 1074 Carbon Steel
All polars? True | 4 Tables | Steel: AISI 1055
All polars? True | 4 Tables | Steel: AISI 4047
All polars? True | 5 Tables | Steel: AISI 1030
All polars? True | 4 Tables | Steel: AISI-SAE 1065
All polars? True | 4 Tables | Steel: AISI 6145
All polars? True | 5 Tables | Steel: AISI 1095
All polars? True | 5 Tables | Steel: AISI-SAE 6150
All polars? True | 4 Tables | Steel: AISI-SAE 1335
All polars? True | 4 Tables | Steel: AISI-SAE 1038
All polars? False | 5 Tables | Steel: AISI-SAE 4340
All polars? True | 3 Tables | Steel: AISI 1049
All polars? True | 4 Tables | Steel: AISI 52100
All polars? True | 4 Tables | Steel: AISI 1090
All polars? True | 5 Tables | Steel: AISI-SAE 1026
A

### Check if all kept tables are good

Confirm all are polars dataframes

Look at the shape of all of them and determine which will be limiting

In [104]:
dict_phys_props = {}
dict_mech_props = {}

polars_check = True 
for steel, tables in dict_steel_tables_keep.items():
    try:
        all_polars = True
        count_polars = 0
        for table in tables:
            if isinstance(table, pl.DataFrame): 
                count_polars += 1 
        if count_polars != len(tables):
            all_polars = False
        if not (bool_check and all_polars):
            bool_check = False
        print("All polars?", all_polars, '|', len(tables), "Tables | Steel:", steel)

        phys_props = tables[0]
        mech_props = tables[1]
        print("Physical properties:", phys_props.shape, "| Properties head(1)", phys_props.select(pl.col('Properties').head(1)) )
        print("Mechanical propsshape:", mech_props.shape, "| Properties head(1)", mech_props.select(pl.col('Properties').head(1)) )
        dict_phys_props[steel] = phys_props
        dict_mech_props[steel] = mech_props
    except:
        dict_phys_props[steel] = pl.DataFrame({'Properties' : [], 'Metric' : []})
        dict_mech_props[steel] = pl.DataFrame({'Properties' : [], 'Metric' : []})

print("All Polars?", polars_check)

All polars? True | 2 Tables | Steel: AISI-SAE 4037
Physical properties: (1, 3) | Properties head(1) shape: (1, 1)
┌────────────┐
│ Properties │
│ ---        │
│ str        │
╞════════════╡
│ Density    │
└────────────┘
Mechanical propsshape: (10, 3) | Properties head(1) shape: (1, 1)
┌─────────────────┐
│ Properties      │
│ ---             │
│ str             │
╞═════════════════╡
│ Elastic modulus │
└─────────────────┘
All polars? True | 2 Tables | Steel: AISI-SAE 1040
Physical properties: (2, 3) | Properties head(1) shape: (1, 1)
┌─────────────────────────────────┐
│ Properties                      │
│ ---                             │
│ str                             │
╞═════════════════════════════════╡
│ Density (chemical composition … │
└─────────────────────────────────┘
Mechanical propsshape: (16, 3) | Properties head(1) shape: (1, 1)
┌──────────────────┐
│ Properties       │
│ ---              │
│ str              │
╞══════════════════╡
│ Tensile strength │
└────────────────

# Filter for complete data, and select unit standard

Kpep only property and metric column from both sets of tables

Find which properties are present in all tables of each type

In [105]:
def remove_imperial(property_dict):
    new_dict = {}
    for key, table in property_dict.items():
        new_dict[key] = table.select(["Properties", "Metric"])
    return new_dict

dict_phys_props =  remove_imperial(dict_phys_props)
dict_mech_props = remove_imperial(dict_mech_props)


dict_phys_props['AISI-SAE 1050']

Properties,Metric
str,str
"""Density""","""7.85 g/cm3"""


# Properties to keep

Only properties present in all metals are the following:

Physical
* Density

Mechanical
* Elastic modulus
* Poisson's ratio

## Process Density into a df

In [106]:
#fix formatting in single cell. Did not incude a space 
# TODO: handle with regex to additional similar errors in additional rows
# arrives as '7.7-8.03g/cm3'
dict_phys_props['AISI 1074 Carbon Steel'][0, "Metric"] = '7.7-8.03 g/cm3'
# arrives as '200GPa'
dict_mech_props['AISI-SAE 1065'][2, "Metric"] = '200 GPa'
dict_mech_props['AISI-SAE 1065']

Properties,Metric
str,str
"""Tensile strength, ultimate""","""635 MPa"""
"""Tensile strength, yield""","""490 MPa"""
"""Modulus of elasticity""","""200 GPa"""
"""Bulk modulus (typical for stee…","""140 GPa"""
"""Shear modulus (typical for ste…","""80 GPa"""
…,…
"""Hardness, Knoop (converted fro…","""209"""
"""Hardness, Rockwell B (converte…","""90"""
"""Hardness, Rockwell C (converte…","""10"""


In [107]:
s_steel = pl.Series(name = 'steel', dtype= pl.String)
s_density = pl.Series(name = 'density', dtype= pl.String)
s_unit = pl.Series(name = 'units_density', dtype= pl.String)

for steel, table in dict_phys_props.items():
    try:
        #select only the  cell of interest
        df_density = table.filter(pl.col('Properties').str.contains("Density"))['Metric'].str.split(' ')
        density = df_density.list.get(0)
        unit = df_density.list.get(1)
        # Make a series of all 3
        s_steel.extend(pl.Series(name = 'steel', values = [steel]))
        s_density = s_density.extend(density)
        s_unit = s_unit.extend(unit)
    except:
        print("Failed on", steel)

Failed on AISI-SAE 9254


In [108]:
df_steel_properties = pl.DataFrame([s_steel, s_density, s_unit])
df_steel_properties

steel,density,units_density
str,str,str
"""AISI-SAE 4037""","""7.85""","""g/cm3"""
"""AISI-SAE 1040""","""7.845""","""g/cc"""
"""AISI-SAE 4140""","""7.85""","""g/cm3"""
"""AISI-SAE 1030""","""7.85""","""g/cc"""
"""AISI-SAE 5160""","""7.85""","""g/cm3"""
…,…,…
"""AISI-SAE 1035""","""7.85""","""g/cm3"""
"""AISI-SAE 1045""","""7.87""","""g/cm3"""
"""AISI-SAE 5140""","""7.85""","""g/cm3"""


## Process mechanical properties

In [109]:
def grab_property(table, property):
    s_result = table.filter(pl.col('Properties')\
                            .str.to_lowercase()\
                                .str.contains(property))\
                                ['Metric']
    return s_result

In [110]:
def get_oobT(df, index):
    return df.list.get(index, null_on_oob=True)

In [111]:
# list of tuples if data was sucessfully scraped from AZoM
azom_sucessful = {'steel':[],'AZoM-Successful':[]}
list_of_scraped = [("steel", "pr", "em", "em_unit", "ys", "ys_unit", "uts", "uts_unit")]

for steel, table in dict_mech_props.items():
    pr = None
    em = None
    em_unit = None
    ys = None
    ys_unit = None
    uts = None
    uts_unit = None

    try:
        # Grab values and split into units where necessary
        #poisson's ratio
        pr = grab_property(table, "poisson")

        #elastic modulus
        df_em = grab_property(table, "elastic").str.split(' ')
        em = get_oobT(df_em, 0)
        em_unit = get_oobT(df_em, 1)
        # em = df_em.list.get(0, null_on_oob=True)
        # em_unit = df_em.list.get(1, null_on_oob=True)

        # grab yield strength
        df_ys = grab_property(table, "yield").str.split(' ')
        ys = get_oobT(df_ys, 0)
        ys_unit = get_oobT(df_ys, 1)

        # grab strength value that is not yield strength.
        # Tensile strength or ultimate yield strength or ultimate tensile strength. All the same measurement
        df_uts = table.filter(pl.col('Properties').str.to_lowercase().str.contains("strength") & 
                            pl.col('Properties').str.to_lowercase().str.contains("yield").not_())\
                                ['Metric'].str.split(' ')
        uts = get_oobT(df_uts, 0)
        uts_unit = get_oobT(df_uts, 1)
    
        # Make a tuple for each row
        # If this fails it will be an IndexError and go to the except
        row = (steel, pr[0], em[0], em_unit[0], ys[0], ys_unit[0], uts[0], uts_unit[0])

        list_of_scraped.append(row)
        azom_sucessful['steel'].append(steel)
        azom_sucessful['AZoM-Successful'].append(True)
        
    except (IndexError, InvalidOperationError):
        print("Failed on", steel)
        azom_sucessful['steel'].append(steel)
        azom_sucessful['AZoM-Successful'].append(False)

df_AZoM_successfull = pl.DataFrame(azom_sucessful)
df_AZoM_successfull.filter(pl.col('AZoM-Successful')==False)

Failed on AISI-SAE 4037
Failed on AISI 4047
Failed on AISI-SAE 1335
Failed on AISI 1049
Failed on AISI 52100
Failed on AISI-SAE 1042
Failed on AISI-SAE 9254
Failed on AISI-SAE 1080


steel,AZoM-Successful
str,bool
"""AISI-SAE 4037""",false
"""AISI 4047""",false
"""AISI-SAE 1335""",false
"""AISI 1049""",false
"""AISI 52100""",false
"""AISI-SAE 1042""",false
"""AISI-SAE 9254""",false
"""AISI-SAE 1080""",false


## Convert to dataframes and save

Checkpoint for continued work in this file so the scraping does not need to be done every time.

In [112]:
df_temp = pl.DataFrame(list_of_scraped, orient='row')
# rename columns using first row
# https://stackoverflow.com/questions/75187317/how-to-rename-column-names-with-first-row-in-polars
# [1:] removes the first row that replaced the column names 
df_temp = df_temp.rename(df_temp.head(1).to_dicts().pop())[1:]
df_temp.tail(3)

steel,pr,em,em_unit,ys,ys_unit,uts,uts_unit
str,str,str,str,str,str,str,str
"""AISI-SAE 1045""","""0.29""","""200""","""GPa""","""450""","""MPa""","""585""","""MPa"""
"""AISI-SAE 5140""","""0.27-0.30""","""190-210""","""GPa""","""295""","""MPa""","""570""","""MPa"""
"""AISI 4640""","""0.27-030""","""201-209""","""GPa""","""1103""","""MPa""","""1276""","""MPa"""


In [113]:
df_steel_properties = df_steel_properties.join(df_temp, on='steel', how='inner')
df_steel_properties.write_csv(f'{resources_path}/AZoM_scraped_properties.csv')
df_steel_properties


steel,density,units_density,pr,em,em_unit,ys,ys_unit,uts,uts_unit
str,str,str,str,str,str,str,str,str,str
"""AISI-SAE 1040""","""7.845""","""g/cc""","""0.27-0.30""","""190-210""","""GPa""","""415""","""MPa""","""620""","""MPa"""
"""AISI-SAE 4140""","""7.85""","""g/cm3""","""0.27-0.30""","""190-210""","""GPa""","""415""","""MPa""","""655""","""MPa"""
"""AISI-SAE 1030""","""7.85""","""g/cc""","""0.27-0.30""","""190-210""","""GPa""","""440""","""MPa""","""525""","""MPa"""
"""AISI-SAE 5160""","""7.85""","""g/cm3""","""0.27 – 0.30""","""190-210""","""GPa""","""275""","""MPa""","""724""","""MPa"""
"""AISI 1074 Carbon Steel""","""7.7-8.03""","""g/cm3""","""0.27-0.30""","""190-210""","""GPa""","""505""","""MPa""","""650""","""MPa"""
…,…,…,…,…,…,…,…,…,…
"""AISI-SAE 1050""","""7.85""","""g/cm3""","""0.27-0.30""","""190-210""","""GPa""","""580""","""MPa""","""690""","""MPa"""
"""AISI-SAE 1035""","""7.85""","""g/cm3""","""0.27-0.30""","""190-210""","""GPa""","""370""","""MPa""","""585""","""MPa"""
"""AISI-SAE 1045""","""7.87""","""g/cm3""","""0.29""","""200""","""GPa""","""450""","""MPa""","""585""","""MPa"""


In [114]:
s_AZOM_successful = df_AZoM_successfull.filter(pl.col('AZoM-Successful')==False)['steel']
pl.DataFrame(s_AZOM_successful).write_csv(f'{resources_path}/MIF_search.csv')

# Process units from scraped AZoM data

Ensure all units are base units and the same down each column

In [115]:
data_path = f"{resources_path}/AZoM_scraped_properties.csv"
df_scraped = pl.read_csv(data_path)
df_scraped.glimpse()

Rows: 21
Columns: 10
$ steel         <str> 'AISI-SAE 1040', 'AISI-SAE 4140', 'AISI-SAE 1030', 'AISI-SAE 5160', 'AISI 1074 Carbon Steel', 'AISI 1055', 'AISI 1030', 'AISI-SAE 1065', 'AISI 6145', 'AISI 1095'
$ density       <str> '7.845', '7.85', '7.85', '7.85', '7.7-8.03', '7.85', '7.85', '7.85', '7.75', '7.85'
$ units_density <str> 'g/cc', 'g/cm3', 'g/cc', 'g/cm3', 'g/cm3', 'g/cm3', 'g/cc', 'g/cm3', 'g/cm3', 'g/cm3'
$ pr            <str> '0.27-0.30', '0.27-0.30', '0.27-0.30', '0.27 – 0.30', '0.27-0.30', '0.27-0.30', '0.27-0.30', '0.27-0.30', '0.27-030', '0.27-0.30'
$ em            <str> '190-210', '190-210', '190-210', '190-210', '190-210', '190-210', '190-210', '200', '201-209', '190-210'
$ em_unit       <str> 'GPa', 'GPa', 'GPa', 'GPa', 'GPa', 'GPa', 'GPa', 'GPa', 'GPa', 'GPa'
$ ys            <i64> 415, 415, 440, 275, 505, 560, 440, 490, 1165, 525
$ ys_unit       <str> 'MPa', 'MPa', 'MPa', 'MPa', 'MPa', 'MPa', 'MPa', 'MPa', 'MPa', 'MPa'
$ uts           <i64> 620, 655, 525, 724, 650, 6

In [116]:
df_scraped.select(['units_density', 'em_unit', "ys_unit", "uts_unit"]).unique()

units_density,em_unit,ys_unit,uts_unit
str,str,str,str
"""g/cc""","""GPa""","""MPa""","""MPa"""
"""g/""","""GPa""","""MPa""","""MPa"""
"""g/cm3""","""GPa""","""MPa""","""MPa"""


## All units are valid and ready to be added to header and unit column removed

All units in units_density are a 1:1 conversion with g/cm3

g/mL (water) = g/cc = g/cm3 = grams per cubic centimeter

Converns:
* emdash and hyphens present
* AISI-SAE 1065 did not separate from it's decimal number.
* AISI-SAE 1026 indicatse g/ but was originally "g/ cm3"

In [117]:
df_scraped_unit_header = df_scraped.select(['steel', 
                                            'density', 
                                            'pr', 
                                            'em', 
                                            'ys',
                                            'uts']).rename({'density' : 'density_g_per_cm3',
                                                                        'em' : 'em_GPa',
                                                                        'ys' : 'ys_MPa',
                                                                        'uts' : 'uts_MPa'})

print(df_scraped_unit_header.shape)    
df_scraped_unit_header.head()

(21, 6)


steel,density_g_per_cm3,pr,em_GPa,ys_MPa,uts_MPa
str,str,str,str,i64,i64
"""AISI-SAE 1040""","""7.845""","""0.27-0.30""","""190-210""",415,620
"""AISI-SAE 4140""","""7.85""","""0.27-0.30""","""190-210""",415,655
"""AISI-SAE 1030""","""7.85""","""0.27-0.30""","""190-210""",440,525
"""AISI-SAE 5160""","""7.85""","""0.27 – 0.30""","""190-210""",275,724
"""AISI 1074 Carbon Steel""","""7.7-8.03""","""0.27-0.30""","""190-210""",505,650


In [118]:
df_scraped_unit_header.describe()

statistic,steel,density_g_per_cm3,pr,em_GPa,ys_MPa,uts_MPa
str,str,str,str,str,f64,f64
"""count""","""21""","""21""","""21""","""21""",21.0,21.0
"""null_count""","""0""","""0""","""0""","""0""",0.0,0.0
"""mean""",null,null,null,null,508.47619,680.190476
"""std""",null,null,null,null,223.223569,200.888183
"""min""","""AISI 1030""","""7.7-8.03""","""0.27 – 0.30""","""190-210""",275.0,490.0
"""25%""",null,null,null,null,415.0,570.0
"""50%""",null,null,null,null,450.0,650.0
"""75%""",null,null,null,null,525.0,690.0
"""max""","""AISI-SAE 6150""","""7.87""","""0.29""","""201-209""",1165.0,1276.0


## Process ranges to an average value

### Helper functions for mapping

In [119]:
df_scraped_clean = df_scraped_unit_header.clone()


In [120]:
# string to number conversion that is aware all values are decimal, but not all have decimal points 
def if_no_decimal_point(num):
    #If leading 0 followed by number then replace leading 0 with "0."
    regex = r"^0\d"
    if re.search(regex, num):
        #replace the 
        num = re.sub("^0", "0.", num)
    return np.float64(num)

def strip_string(string):
    return string.strip()

def process_ranged_val_cleaned(col, df=df_scraped_clean):
    # AISI-SAE 5160 and AISI-SAE 6150 use a different dash than the others for the pr range
    # Discovered when 2 nulls showed up after processing
        # mean could not process the string and resulted in a null
    s_same_dashes = df[col].str.replace('–', '-')
    # Split the range froma string into two strings of numbers
    s_split = s_same_dashes.str.split('-')
    # Convert these decimal points to numbers 
        #aware that some decimal values are formatted 0## with no .
    s_numeric_l = s_split.map_elements(lambda x: list(map(if_no_decimal_point, x)))
    # take middle of the two values
    return s_numeric_l.list.mean()

# Process Units 

In [123]:
# Split the range froma string into two strings of numbers
s_split = df_scraped_clean['density_g_per_cm3'].str.split('-')
# Convert these decimal points to numbers 
s_numeric_l = s_split.map_elements(lambda x: list(map(if_no_decimal_point, x)))
# take middle of the two values
s_mean_d = s_numeric_l.list.mean()

/tmp/ipykernel_139906/1347279945.py:4: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  s_numeric_l = s_split.map_elements(lambda x: list(map(if_no_decimal_point, x)))


In [131]:
s_mean_p = process_ranged_val_cleaned("pr")
s_mean_em = process_ranged_val_cleaned("em_GPa")

/tmp/ipykernel_139906/149081071.py:22: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  s_numeric_l = s_split.map_elements(lambda x: list(map(if_no_decimal_point, x)))


In [127]:
df_clean = df_scraped_clean.with_columns(poissons_ratio = s_mean_p, 
                                         density_g_per_cm3 = s_mean_d, 
                                         elastic_modulus_GPa = s_mean_em,
                                         yield_strength_MPa = df_scraped_unit_header['ys_MPa'],
                                         ultimate_strength_MPA = df_scraped_unit_header['uts_MPa'])\
                                            .select(['steel',
                                                     "density_g_per_cm3", 
                                                     "poissons_ratio",
                                                     "elastic_modulus_GPa", 
                                                     "yield_strength_MPa",
                                                     "ultimate_strength_MPA"])
df_clean.describe()

statistic,steel,density_g_per_cm3,poissons_ratio,elastic_modulus_GPa,yield_strength_MPa,ultimate_strength_MPA
str,str,f64,f64,f64,f64,f64
"""count""","""21""",21.0,21.0,21.0,21.0,21.0
"""null_count""","""0""",0.0,0.0,0.0,0.0,0.0
"""mean""",null,7.842048,0.285238,200.47619,508.47619,680.190476
"""std""",null,0.031154,0.001091,1.503963,223.223569,200.888183
"""min""","""AISI 1030""",7.75,0.285,200.0,275.0,490.0
"""25%""",null,7.85,0.285,200.0,415.0,570.0
"""50%""",null,7.85,0.285,200.0,450.0,650.0
"""75%""",null,7.85,0.285,200.0,525.0,690.0
"""max""","""AISI-SAE 6150""",7.87,0.29,205.0,1165.0,1276.0


In [130]:
df_clean.write_csv(f'{resources_path}/AZoM_scraped_properties.csv')

## Scraping MakeItFrom.com